## 🏁 Notebook 1: Data Cleaning & Preparation

### 🎯 Objective
In this notebook, we clean, merge, and prepare the raw Formula 1 datasets (1950–2025) to create a consolidated, analysis-ready dataset for the **Team Dominance Index (TDI)** project.

By the end of this notebook, we’ll have a processed file:


In [12]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("✅ Libraries imported successfully.")

✅ Libraries imported successfully.


In [13]:
RAW_DATA_PATH = "../Data/raw/"
PROCESSED_DATA_PATH = "../Data/processed/"

print("📂 Files found in raw data folder:")
os.listdir(RAW_DATA_PATH)

📂 Files found in raw data folder:


['f1db-constructors.csv',
 'f1db-races-constructor-standings.csv',
 'f1db-races-qualifying-1-results.csv',
 'f1db-races-race-results.csv',
 'f1db-races.csv']

In [14]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')


RAW_DATA_PATH = "../Data/raw/"

results = pd.read_csv(RAW_DATA_PATH + "f1db-races-race-results.csv")
constructors = pd.read_csv(RAW_DATA_PATH + "f1db-constructors.csv")
standings = pd.read_csv(RAW_DATA_PATH + "f1db-races-constructor-standings.csv")  # ✅ fixed filename

try:
    qualifying = pd.read_csv(RAW_DATA_PATH + "f1db-races-qualifying-1-results.csv")
except FileNotFoundError:
    qualifying = None
    print("⚠️ Qualifying data not found — continuing without it.")

races = pd.read_csv(RAW_DATA_PATH + "f1db-races.csv")

print("✅ Data files successfully loaded!")
print(f"Results shape: {results.shape}")
print(f"Constructors shape: {constructors.shape}")
print(f"Standings shape: {standings.shape}")
print(f"Races shape: {races.shape}")
if qualifying is not None:
    print(f"Qualifying shape: {qualifying.shape}")

✅ Data files successfully loaded!
Results shape: (27211, 34)
Constructors shape: (185, 19)
Standings shape: (10449, 10)
Races shape: (1149, 42)
Qualifying shape: (7707, 24)


In [6]:
print("=== RESULTS ===")
print(results.columns.tolist())

print("\n=== STANDINGS ===")
print(standings.columns.tolist())

print("\n=== CONSTRUCTORS ===")
print(constructors.columns.tolist())

print("\n=== RACES ===")
print(races.columns.tolist())

print("\n=== QUALIFYING ===")
print(qualifying.columns.tolist())


=== RESULTS ===
['raceId', 'year', 'round', 'positionDisplayOrder', 'positionNumber', 'positionText', 'driverNumber', 'driverId', 'constructorId', 'engineManufacturerId', 'tyreManufacturerId', 'sharedCar', 'laps', 'time', 'timeMillis', 'timePenalty', 'timePenaltyMillis', 'gap', 'gapMillis', 'gapLaps', 'interval', 'intervalMillis', 'reasonRetired', 'points', 'polePosition', 'qualificationPositionNumber', 'qualificationPositionText', 'gridPositionNumber', 'gridPositionText', 'positionsGained', 'pitStops', 'fastestLap', 'driverOfTheDay', 'grandSlam']

=== STANDINGS ===
['raceId', 'year', 'round', 'positionDisplayOrder', 'positionNumber', 'positionText', 'constructorId', 'engineManufacturerId', 'points', 'positionsGained']

=== CONSTRUCTORS ===
['id', 'name', 'fullName', 'countryId', 'bestChampionshipPosition', 'bestStartingGridPosition', 'bestRaceResult', 'totalChampionshipWins', 'totalRaceEntries', 'totalRaceStarts', 'totalRaceWins', 'total1And2Finishes', 'totalRaceLaps', 'totalPodiums',

In [15]:
# --- RESULTS---
results_clean = results[[
    'raceId', 'year', 'round',
    'positionDisplayOrder', 'positionNumber', 'positionText',
    'constructorId', 'engineManufacturerId',
    'points', 'positionsGained'
]].rename(columns={
    'positionDisplayOrder': 'finish_position'
})


# --- STANDINGS---
standings_clean = standings[[
    'raceId', 'constructorId', 'points', 'positionDisplayOrder'
]].rename(columns={
    'positionDisplayOrder': 'standings_position'
})


# --- CONSTRUCTORS---
constructors_clean = constructors[[
    'id', 'name', 'countryId', 'totalRaceWins', 
    'totalChampionshipWins', 'totalPoints'
]].rename(columns={
    'id': 'constructorId',
    'countryId': 'nationality'
})


# --- RACES---
races_clean = races[[
    'id', 'year', 'round', 'officialName', 'circuitId', 'distance', 'laps'
]].rename(columns={
    'id': 'raceId',
    'officialName': 'race_name'
})


print("✅ Clean datasets prepared successfully!")
print(f"Results_clean: {results_clean.shape}")
print(f"Standings_clean: {standings_clean.shape}")
print(f"Constructors_clean: {constructors_clean.shape}")
print(f"Races_clean: {races_clean.shape}")

✅ Clean datasets prepared successfully!
Results_clean: (27211, 10)
Standings_clean: (10449, 4)
Constructors_clean: (185, 6)
Races_clean: (1149, 7)


In [16]:
merged_df = (
    results_clean
    .merge(standings_clean, on=["raceId", "constructorId"], how="left")
    .merge(constructors_clean, on="constructorId", how="left")
    .merge(races_clean, on=["raceId", "year", "round"], how="left")
)

print("✅ Merged master dataset created successfully!")
print("Shape:", merged_df.shape)
print("\nColumns:\n", merged_df.columns.tolist())

✅ Merged master dataset created successfully!
Shape: (27797, 21)

Columns:
 ['raceId', 'year', 'round', 'finish_position', 'positionNumber', 'positionText', 'constructorId', 'engineManufacturerId', 'points_x', 'positionsGained', 'points_y', 'standings_position', 'name', 'nationality', 'totalRaceWins', 'totalChampionshipWins', 'totalPoints', 'race_name', 'circuitId', 'distance', 'laps']


In [17]:
merged_df = merged_df.rename(columns={
    'points_x': 'race_points',
    'points_y': 'champ_points'
})

print("✅ Columns renamed for clarity!")
print(merged_df.columns.tolist())

✅ Columns renamed for clarity!
['raceId', 'year', 'round', 'finish_position', 'positionNumber', 'positionText', 'constructorId', 'engineManufacturerId', 'race_points', 'positionsGained', 'champ_points', 'standings_position', 'name', 'nationality', 'totalRaceWins', 'totalChampionshipWins', 'totalPoints', 'race_name', 'circuitId', 'distance', 'laps']


In [18]:
import numpy as np

tdi = (
    merged_df.groupby(['year', 'constructorId', 'name'])
    .agg(
        races=('raceId', 'nunique'),
        wins=('finish_position', lambda x: (x == 1).sum()),
        podiums=('finish_position', lambda x: (x <= 3).sum()),
        avg_finish=('finish_position', 'mean'),
        total_points=('race_points', 'sum'),
    )
    .reset_index()
)

# Derived metrics
tdi['win_rate'] = tdi['wins'] / tdi['races']
tdi['podium_rate'] = tdi['podiums'] / tdi['races']
tdi['avg_points'] = tdi['total_points'] / tdi['races']

# Normalize and compute TDI (weighted sum)
tdi['TDI'] = (
    (tdi['win_rate'] * 0.4) +
    (tdi['podium_rate'] * 0.25) +
    ((1 / tdi['avg_finish']) * 0.1) +   # inverse because lower finish = better
    (tdi['avg_points'] / tdi['avg_points'].max() * 0.25)
)

print("✅ Team Dominance Index calculated!")
print(tdi.head(10))

✅ Team Dominance Index calculated!
   year constructorId          name  races  wins  podiums  avg_finish  total_points  win_rate  podium_rate  avg_points       TDI
0  1950         adams         Adams      1     0        0   29.500000           0.0       0.0     0.000000    0.000000  0.003390
1  1950    alfa-romeo    Alfa Romeo      6     6       11    6.954545          89.0       1.0     1.833333   14.833333  0.972938
2  1950          alta          Alta      2     0        0   13.333333           0.0       0.0     0.000000    0.000000  0.007500
3  1950        cooper        Cooper      1     0        0   15.000000           0.0       0.0     0.000000    0.000000  0.006667
4  1950         deidt         Deidt      1     0        2   13.000000          10.0       0.0     2.000000   10.000000  0.575260
5  1950           era           ERA      3     0        0   14.125000           0.0       0.0     0.000000    0.000000  0.007080
6  1950         ewing         Ewing      1     0        0   18

In [19]:
tdi.replace([np.inf, -np.inf], np.nan, inplace=True)
tdi.fillna(0, inplace=True)

numeric_cols = ['races', 'wins', 'podiums', 'avg_finish', 'total_points',
                'win_rate', 'podium_rate', 'avg_points', 'TDI']
tdi[numeric_cols] = tdi[numeric_cols].apply(pd.to_numeric, errors='coerce')

output_path = "../Data/processed/team_dominance_index.csv"
tdi.to_csv(output_path, index=False)
print(f"Saved processed data to {output_path}")
print(f"Shape: {tdi.shape}")
print("Columns:", list(tdi.columns))

Saved processed data to ../Data/processed/team_dominance_index.csv
Shape: (1065, 12)
Columns: ['year', 'constructorId', 'name', 'races', 'wins', 'podiums', 'avg_finish', 'total_points', 'win_rate', 'podium_rate', 'avg_points', 'TDI']
